# Solve a Generalized Assignment Problem using Lagrangian relaxation

This tutorial includes data and information that you need to set up Decision Optimization engines and build mathematical programming models to solve a Generalized Assignment Problem using Lagrangian relaxation.


When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of the [Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)

>You will need a valid subscription to Decision Optimization on Cloud ([here](https://developer.ibm.com/docloud)) or a local installation of CPLEX Optimizers. 

Some familiarity with Python is recommended. This notebook runs on Python 2.7 and 3.5.

## Table of contents
* [Describe the business problem](#describe-problem)
* [How Decision Optimization can help](#do-help) 
* [Use Decision Optimization to create and solve the model](#do-model-create-solve)
   1. [Download the library](#Download-the-library)<br>
   2. [Set up the prescriptive engine](#Set-up-the-prescriptive-engine)<br>
   3. [Model the Data](#Step-3:-Model-the-data)<br>
   4. [Set up the prescriptive model](#Set-up-the-prescriptive-model)<br>
      4.1 [Create the DOcplex model](#create-model)<br>
      4.2 [Define the decision variables](#Define-the-decision-variables)<br>
      4.3 [Define the business constraints](#Express-the-business-constraints)<br>
      4.4 [Solve the model by using the Decision Optimization solve service](#solve-model-solve-service)<br>
      4.5 [Solve the model by using the Lagrangian Relaxation method](#solve-model-lagrangian-relaxation)<br>
   5. [Investigate the solution and run an example analysis](#Investigate-the-solution)<br>
* [Summary](#summary)<br>  

<a id="describe-problem"></a>
## Describe the business problem   


This notebook illustrates how to solve an optimization model using Lagrangian relaxation technics. 
It solves a generalized assignment problem (GAP), as defined by Wolsey, using this relaxation technic.

The main aim is to show multiple optimizations through modifications of different models that exist in a single environment; not to show how to solve a GAP problem.

In the field of Mathematical Programming, this technic is based on an approximation of a difficult, constrained problem to a simpler problem: you remove difficult constraints by integrating them in the objective function, penalizing the objective if a constraint is not respected.

This method penalizes violations of inequality constraints by using a Lagrange multiplier that imposes a cost on violations. The added costs are used instead of the strict inequality constraints in the optimization. 

In practice, the relaxed problem can often be solved more easily than the original problem.

For more information, see the following Wikipedia articles: [Generalized assignment problem](https://en.wikipedia.org/wiki/Generalized_assignment_problem) and [Lagrangian relaxation](https://en.wikipedia.org/wiki/Lagrangian_relaxation).

This notebook first solves the standard problem, which is not important here, then shows how to reformulate the problem to meet the Lagrangian Relaxation features.

<a id="do-help"></a>
## How  decision optimization can help

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It considers specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control over business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes. Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle a future situation. Organizations that act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.

With prescriptive analytics, you can:

* Automate the complex decisions and trade-offs to better manage your limited resources.
    
* Take advantage of a future opportunity or mitigate a future risk.
    
* Proactively update recommendations based on changing events.
    
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.





<a id="do-model-create-solve"></a>
## Use Decision Optimization to create and solve the model


<a id="Download-the-library"></a>
### 1. Download the library

Run the following code to install the Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains two modeling packages, mathematical programming (docplex.mp) package and constraint programming (docplex.cp) package.

In the following code, `real.prefix` is used to ensure that the script is not running inside a virtual environment.

In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex 
    else:
        !pip install --user docplex      

<a id="Set-up-the-prescriptive-engine"></a>
### 2. Set up the prescriptive engine

To solve DOcplex models, you need access to the DOcplexcloud solve service.

* Subscribe to the [Decision Optimization on Cloud](https://developer.ibm.com/docloud) (DOcplexcloud) service.
* Get the service base URL and personal API key.
* Enter the URL and API key in the cell below, enclosed in quotation marks (""), and run the cell.

In [2]:
url = None
key = None

<a id="Step-3:-Model-the-data"></a>
### 3. Model the data
In this scenario, the data is simple. It is delivered as 3 input arrays: A, B, and C. The data does not need changing or refactoring.

In [3]:
B = [15, 15, 15]
C = [
    [ 6, 10, 1],
    [12, 12, 5],
    [15,  4, 3],
    [10,  3, 9],
    [8,   9, 5]
]
A = [
    [ 5,  7,  2],
    [14,  8,  7],
    [10,  6, 12],
    [ 8,  4, 15],
    [ 6, 12,  5]
]

<a id="Set-up-the-prescriptive-model"></a>
### 4. Set up the prescriptive model

Start with viewing the environment information. This information should be updated when you run the notebook.
 

In [4]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python is present, version is 3.5.2
* docplex is present, version is (2, 0, 15)
* CPLEX wrapper is present, version is 12.7.0.0, located at: /opt/conda/lib/python3.5/site-packages


We will first create an optimization problem that is composed of 2 basic constraint blocks. Then, we will resolve the problem using Lagrangian Relaxation on 1 of the constraint blocks.

<a id="create-model"></a>
#### 4.1 Create the DOcplex model
The model contains the business constraints and the objective.


In [5]:
from docplex.mp.model import Model

mdl = Model("GAP")

<a id="Define-the-decision-variables"></a>
#### 4.2 Define the decision variables

In [6]:
print("#As={}, #Bs={}, #Cs={}".format(len(A), len(B), len(C)))
number_of_cs = len(C)
# variables
x_vars = [mdl.binary_var_list(c, name=None) for c in C]

#As=5, #Bs=3, #Cs=5


<a id="Express-the-business-constraints"></a>
#### 4.3 Define the business constraints

In [7]:
# constraints
cts = mdl.add_constraints(mdl.sum(xv) <= 1 for xv in x_vars)

mdl.add_constraints(mdl.sum(x_vars[ii][j] * A[ii][j] for ii in range(number_of_cs)) <= bs for j, bs in enumerate(B))

# objective
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.maximize(total_profit)
mdl.print_information()

Model: GAP
 - number of variables: 15
   - binary=15, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults


<a id="solve-model-solve-service"></a>
#### 4.4. Solve the model by using the Decision Optimization solve service

Use the Decision Optimization on Cloud solve service to solve the model or use a local engine to solve the model. 

In [8]:
s = mdl.solve(url=url, key=key, log_output=True)
assert s is not None
obj = s.objective_value
print("* GAP with no relaxation run OK, best objective is: {:g}".format(obj))

Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 7 coefficients.
Reduced MIP has 8 rows, 15 columns, and 30 nonzeros.
Reduced MIP has 15 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 8 rows, 15 columns, and 30 nonzeros.
Reduced MIP has 15 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 15.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000      112.0000              --- 
*     0+    0 

<a id="solve-model-lagrangian-relaxation"></a>
#### 4.5. Solve the model by using the Lagrangian Relaxation method

To demonstrate the Lagrangian Relaxation method, let's consider that this model was hard to solve for CPLEX.
We will approximate this problem by creating an iterative model where the objective is modified at each iteration. 

**Tip**: Wait a few seconds for the solution (due to a time limit parameter). 

First, we need to remove the culprit constraints from the model.

In [9]:
for ct in cts:
    mdl.remove_constraint(ct)

As these constraints are removed, we need to ensure that they are satisfied by introducing new _penalization_ variables that hold the slack values of the constraints, and add the variables, with increased weight, to the objective.

In [10]:
#p_vars are the penalties attached to violating the constraints
p_vars = mdl.continuous_var_list(C, name='p')  # new for relaxation

In [11]:
# new version of the approximated constraint where we apply the penalties
mdl.add_constraints(mdl.sum(xv) == 1 - pv for xv, pv in zip(x_vars, p_vars));

In [12]:
#Define the maximum number of iterations
max_iters = 10

# define a new kpi
total_profit = mdl.sum(mdl.scal_prod(x_i, c_i) for c_i, x_i in zip(C, x_vars))
mdl.add_kpi(total_profit, "Total profit")


number_of_cs = len(C)
c_range = range(number_of_cs)

#### Lagrangian relaxation main loop

Lagrangian relaxation proceeds with a loop, with a maximum number of iterations. At each iteration, we want to:

- solve the model, maximizing the sum of profit + penalty
- if the constraints are satisfied, within a tolerance of 1e-6, stop
- else, update penalization weights and solve again

The algorithm stops when either all constraints are satisfied or the maximum number of iterations is reached.

In [13]:
# Langrangian relaxation main loop 
eps = 1e-6
loop_count = 0
best = 0
initial_multiplier = 1
multipliers = [initial_multiplier] * len(C)

print("* starting the loop, with a maximum {0} iterations".format(max_iters))
while loop_count <= max_iters:
    loop_count += 1
    # Rebuilt at each loop iteration
    total_penalty = mdl.scal_prod(p_vars, multipliers)
    
    mdl.maximize(total_profit + total_penalty)
    s = mdl.solve(url=url, key=key)
    if not s:
        print("*** solve fails, stopping at iteration: %d" % loop_count)
        break
    best = s.objective_value
    penalties = [pv.solution_value for pv in p_vars]
    print('%d> new lagrangian iteration:\n\t obj=%g, m=%s, p=%s' % (loop_count, best, str(multipliers), str(penalties)))

    do_stop = True
    justifier = 0
    for k in c_range:
        penalized_violation = penalties[k] * multipliers[k]
        if penalized_violation >= eps:
            do_stop = False
            justifier = penalized_violation
            break

    if do_stop:
        print("* Lagrangian relaxation succeeds, best={:g}, penalty={:g}, #iterations={}"
                .format(best, total_penalty.solution_value, loop_count))
        break
    else:
        # Update multipliers and start the loop again.
        scale_factor = 1.0 / float(loop_count)
        multipliers = [max(multipliers[i] - scale_factor * penalties[i], 0.) for i in c_range]
        print('{0}> -- loop continues, m={1!s}, justifier={2:g}'.format(loop_count, multipliers, justifier))

* starting the loop, with a maximum 10 iterations
1> new lagrangian iteration:
	 obj=47, m=[1, 1, 1, 1, 1], p=[0, 0, 0, 0, 1.0]
1> -- loop continues, m=[1.0, 1.0, 1.0, 1.0, 0.0], justifier=1
2> new lagrangian iteration:
	 obj=46, m=[1.0, 1.0, 1.0, 1.0, 0.0], p=[0, 0, 0, 0, 1.0]
* Lagrangian relaxation succeeds, best=46, penalty=0, #iterations=2


In [14]:
print('* optimal result of lagrangian relaxation is : {0}, obtained in {1} iterations'.format(best, loop_count))

* optimal result of lagrangian relaxation is : 46.0, obtained in 2 iterations


<a id="Investigate-the-solution"></a>
### 6. Investigate the solution and run an example analysis

You can see that with this relaxation method applied to our simple model we find the same solution to the problem as with the initial model.


This example also demonstrates the use of Python to program complex workflows.

## Summary


You learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to create a Constraint Programming model and solve it with IBM Decision Optimization on Cloud.

## References
* [CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* [Decision Optimization documentation](https://datascience.ibm.com/docs/content/DO/DOinDSX.html)
* For help with DOcplex, or to report a defect, go [here](https://developer.ibm.com/answers/smartspace/docloud).
* Contact us at dofeedback@wwpdl.vnet.ibm.com


<div class="alert alert-block alert-info"> Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.